## 7.1. 통계적 가설 검정
- 고전적인 가설검정에서는 어떤 기본적인 가설을 의미하는 귀무가설(H_0, null hypothesis)과 비교하고 싶은 대립가설(H_1, alternative hypothesis)로 구성되어 있으며, 통계를 사용해서 H_0을 기각할지 말지를 결정한다.

## 7.2. 예시: 동전 던지기
- 동전에서 앞면이 나올 확률이 p라고 하면, 동전이 공평하다는 의미의 'p=0.5이다'는 귀무가설이 되고, 'p≠0.5이다'는 대립가설이 된다.
- 동전을 n번 던져서 앞면이 나온 횟수 X를 세는 것으로 검정을 진행해 보자.<br> 동전 던지기는 각각 베르누이 분포를 따를 것이며, 이는 X가 이항분포를 따르는 확률변수라는 것을 의미한다.
- 제1종 오류 :  비록 H_0가 참이지만 H_0를 기각하는 'false positive(가양성)' 오류
- 유의수준(significance) : 제1종 오류를 얼마나 허용해 줄 것인지를 의미, 보통 5%나 1%로 설정하는 경우가 많다.
- 제2종 오류 : H_0가 거짓이지만 H_0를 기각하지 않는 오류를 의미하기 때문에, 제2종 오류를 측정하기 위해서는 먼저 H_0가 거짓이라는 것이 무엇을 의미하는지 알아볼 필요가 있다.
- 검정력(power) : 제2종 오류를 범하지 않을 확률

In [7]:
def normal_approximation_to_binomial(n, p):
    '''Binomial(n, p)에 해당되는 mu(평균)와 sigma(표준편차) 계산'''
    mu = p * n
    sigma = math.sqrt(p * (1 - p) * n)
    return mu, sigma

In [8]:
# ch06에서 사용한 정규분포의 누적분포함수
def normal_cdf(x, mu=0, sigma=1):
    return (1 + math.erf((x - mu) / math.sqrt(2) / sigma)) / 2

In [9]:
# 확률변수가 정규분포를 따른다는 가정하에, 
# normal_cdf를 사용하면 실제 동전 던지기로부터 얻은 값이 구간 안(혹은 밖)에 존재할 확률을 계산할 수 있다.

# 누적분포함수는 확률변수가 특정 값보다 작을 확률을 나타낸다.
normal_probability_below = normal_cdf

# 만약 확률변수가 특정 값보다 작지 않다면, 특정 값보다 크다는 것을 의미한다.
def normal_probability_above(lo, mu=0, sigma=1):
    return 1 - normal_cdf(lo, mu, sigma)

# 만약 확률변수가 hi보다 작고 lo보다 작지 않으면, 확률변수는 hi와 lo 사이에 존재한다.
def normal_probability_between(lo, hi, mu=0, sigma=1):
    return normal_cdf(hi, mu, sigma) - normal_cdf(lo, mu, sigma)

# 만약 확률변수가 범위 밖에 존재한다면, 범위 안에 존재하지 않다는 것을 의미한다.
def normal_probability_outside(lo, hi, mu=0, sigma=1):
    return 1 - normal_probability_between(lo, hi, mu, sigma)

In [10]:
# 반대로, 확률이 주어졌을 때 평균을 중심으로 하는 (대칭적인) 구간을 구할 수도 있다.

def normal_upper_bound(probability, mu=0, sigma=1):
    '''P(Z <= z) = probability인 z값을 반환'''
    return inverse_normal_cdf(probability, mu, sigma)

def normal_lower_bound(probability, mu=0, sigma=1):
    '''P(Z >= z) = probability인 z값을 반환'''
    return inverse_normal_cdf(1 - probability, mu, sigma)

def normal_two_sided_bounds(probability, mu=0, sigma=1):
    '''입력한 probability 값을 포함하고, 평균을 중심으로 대칭적인 구간을 반환'''
    tail_probability = (1 - probability) / 2
    
    # 구간의 상항은 tail_probability 값 이상의 확률 값을 갖고 있다.
    upper_bound = normal_lower_bound(tail_probability, mu, sigma)
    
    # 구간의 하한은 tail_probability 값 이하의 확률 값을 갖고 있다.
    lower_bound = normal_upper_bound(tail_probability, mu, sigma)
    
    return lower_bound, upper_bound

In [15]:
# ch06에서 사용한 역함수
def inverse_normal_cdf(p, mu=0, sigma=1, tolerance=0.00001):
    '''이진 검색을 사용해서 역함수를 근사'''
    # 표준정규분포가 아니라면 표준정규분포로 변환
    if mu != 0 or sigma != 1:
        return mu + sigma * inverse_normal_cdf(p, tolerance=tolerance)
    
    low_z, low_p = -10.0, 0  # normal_cdf(-10)는 0에 근접
    hi_z, hi_p = 10.0, 1  # normal_cdf(10)는 1에 근접
    while hi_z - low_z > tolerance:
        mid_z = (low_z + hi_z) / 2  # 중간 값
        mid_p = normal_cdf(mid_z)  # 중간 값의 누적분포 값을 계산
        if mid_p < p:
            # 중간 값이 너무 작다면 더 큰 값들을 검색
            low_z, low_p = mid_z, mid_p
        elif mid_p > p:
            # 중간 값이 너무 크다면 더 작은 값들을 검색
            hi_z, hi_p = mid_z, mid_p
        else:
            break
    return mid_z

# 위의 함수는 원하는 확률 값에 가까워질 때까지 표준정규분포의 구간을 반복적으로 이등분한다.

In [16]:
mu_0, sigma_0 = normal_approximation_to_binomial(1000, 0.5)

normal_two_sided_bounds(0.95, mu_0, sigma_0)

(469.01026640487555, 530.9897335951244)

In [19]:
# p가 0.5라고 가정할 때, 유의수준이 5%인 구간
lo, hi = normal_two_sided_bounds(0.95, mu_0, sigma_0)

# p = 0.55인 경우의 실제 평균과 표준편차
mu_1, sigma_1 = normal_approximation_to_binomial(1000, 0.55)

# 제2종 오류란 귀무가설(H_0)을 기각하지 못한다는 의미
# 즉, X가 주어진 구간 안에 존재할 경우를 의미
type_2_probability = normal_probability_between(lo, hi, mu_1, sigma_1)
power = 1 - type_2_probability
power

0.8865480012953671

In [20]:
hi = normal_upper_bound(0.95, mu_0, sigma_0)
hi  # 526 (<531, 분포 상위 부분에 더 높은 확률을 주기 위해서)

526.0073585242053

In [21]:
type_2_probability = normal_probability_below(hi, mu_1, sigma_1)
power = 1 - type_2_probability
power 

0.9363794803307173

## 7.3. p-value
- 어떤 확률 값을 기준으로 구간을 선택하는 대신에, H_0가 참이라고 가정하고 실제로 관측된 값보다 더 극단적인 값이 나올 확률을 구하는 것.

In [26]:
def two_sided_p_value(x, mu=0, sigma=1):
    if x >= mu:
        # 만약 x가 평균보다 크다면, x보다 큰 부분이 꼬리다
        return 2 * normal_probability_above(x, mu, sigma)
    else:
        # 만약 x가 평균보다 작다면, x보다 작은 부분이 꼬리다
        return 2 * normal_probability_below(x, mu, sigma)
    
print('529.5:',two_sided_p_value(529.5, mu_0, sigma_0))  # 0.06207721579598835
print('530:', two_sided_p_value(530, mu_0, sigma_0)) # 0.05777957112359733
# 연속수정(continuity correction)때문에 530 대신 529.5가 더 정확하다.

529.5: 0.06207721579598835
530: 0.05777957112359733
